In [2]:
# ===============================
# STEP 1: INSTALL REQUIRED PACKAGES
# ===============================
import sys
!{sys.executable} -m pip install pandas requests beautifulsoup4 html5lib lxml --upgrade --quiet


# ===============================
# STEP 2: IMPORT LIBRARIES
# ===============================
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from io import StringIO


# ===============================
# STEP 3: SCRAPE WIKIPEDIA PAGE
# ===============================
url = "https://en.wikipedia.org/wiki/List_of_cities_by_average_temperature"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

response = requests.get(url, headers=headers)
response.raise_for_status()
html = response.text


# ===============================
# STEP 4: PARSE HTML
# ===============================
soup = BeautifulSoup(html, "html.parser")

# Wikipedia tables use class="wikitable"
wiki_tables = soup.find_all("table", class_="wikitable")


# ===============================
# STEP 5: EXTRACT TABLES SAFELY
# ===============================
tables = []
for table in wiki_tables:
    df = pd.read_html(StringIO(str(table)))[0]
    tables.append(df)


# ===============================
# STEP 6: COMBINE ONLY CITY TABLES
# ===============================
combined_df = pd.concat(
    [df for df in tables if "City" in df.columns],
    ignore_index=True
)


# ===============================
# STEP 7: BASIC CLEANING
# ===============================

# Drop reference columns if they exist
combined_df.drop(columns=["Ref", "Ref."], errors="ignore", inplace=True)

# Remove brackets and values inside () or []
combined_df = combined_df.applymap(
    lambda x: re.sub(r"\s*[\(\[].*?[\)\]]", "", x).strip()
    if isinstance(x, str) else x
)


# ===============================
# STEP 8: FIX WEIRD MINUS SIGNS & KEEP NEGATIVES
# ===============================
def clean_numeric_with_minus(x):
    if isinstance(x, str):
        # Fix encoding issues for minus
        x = x.replace("âˆ;", "-").replace("−", "-")
        # Keep only digits, dot, and minus
        cleaned = re.sub(r"[^0-9\.-]", "", x)
        try:
            return float(cleaned)
        except:
            return x
    return x

# Identify month columns
month_cols = [
    col for col in combined_df.columns
    if any(m in col.lower() for m in [
        "jan","feb","mar","apr","may","jun",
        "jul","aug","sep","oct","nov","dec"
    ])
]

# Apply cleaning to month columns
combined_df[month_cols] = combined_df[month_cols].applymap(clean_numeric_with_minus)


# ===============================
# STEP 9: FINAL CHECK
# ===============================
display(combined_df.head())
combined_df.info()


C:\Users\SEGUN\AppData\Local\Temp\ipykernel_20380\2671952336.py:67: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  combined_df = combined_df.applymap(
C:\Users\SEGUN\AppData\Local\Temp\ipykernel_20380\2671952336.py:98: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  combined_df[month_cols] = combined_df[month_cols].applymap(clean_numeric_with_minus)


,Country,City,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
0,Algeria,Algiers,11.2,11.5,13.6,15.6,18.8,22.6,25.7,26.6,23.9,20.4,15.7,12.6,18.2
1,Algeria,Tamanrasset,12.8,15.0,18.1,22.2,26.1,28.9,28.7,28.2,26.5,22.4,17.3,13.9,21.7
2,Algeria,Reggane,16.0,18.2,23.1,27.9,32.2,36.4,39.8,38.4,35.5,29.2,22.0,17.8,28.3
3,Angola,Luanda,26.7,28.5,28.6,28.2,27.0,23.9,22.1,22.1,23.5,25.2,26.7,26.9,25.8
4,Benin,Cotonou,27.3,28.5,28.9,28.6,27.8,26.5,25.8,25.6,26.0,26.7,27.6,27.3,27.2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 479 entries, 0 to 478
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Country  479 non-null    object 
 1   City     479 non-null    object 
 2   Jan      479 non-null    float64
 3   Feb      479 non-null    float64
 4   Mar      479 non-null    float64
 5   Apr      479 non-null    float64
 6   May      479 non-null    float64
 7   Jun      479 non-null    float64
 8   Jul      479 non-null    float64
 9   Aug      479 non-null    float64
 10  Sep      479 non-null    float64
 11  Oct      479 non-null    float64
 12  Nov      479 non-null    float64
 13  Dec      479 non-null    float64
 14  Year     479 non-null    object 
dtypes: float64(12), object(3)
memory usage: 56.3+ KB


In [3]:
file_path = r"C:\Users\SEGUN\Downloads\wikipedia_city_temperature_data.csv"
combined_df.to_csv(file_path, index=False)

print("CSV saved in Downloads folder")


CSV saved in Downloads folder
